In [7]:
import pandas as pd
import plotly.graph_objects as go

In [8]:
df = pd.read_csv('./processed/unigram maps.csv')
df = df.loc[df["Experience"] != "N.A."]

In [9]:
df

,Subject,Experience,Count
0,Social Science and Liberal Arts,Land Revenue Mgmt & District Admn,1736
1,"Governance, Law and Management",Land Revenue Mgmt & District Admn,1227
2,"Engineering, Technology and Mathematics",Land Revenue Mgmt & District Admn,1104
3,Social Science and Liberal Arts,Personnel and General Administration,828
4,Natural Sciences,Land Revenue Mgmt & District Admn,815
...,...,...,...
361,Social Science and Liberal Arts,Anti-Corruption,1
362,Others,Chemicals & Fertilizers,1
363,Others,Mail Management,1
364,Finance,Public Policy,1


In [10]:
max_value = df["Count"].max()

In [11]:
df["Subject"].unique()

array(['Social Science and Liberal Arts',
       'Governance, Law and Management',
       'Engineering, Technology and Mathematics', 'Natural Sciences',
       'Languages and Fine Arts', 'Finance', 'Others', 'Medical'],
      dtype=object)

In [12]:
colors = {}

lower = 100

for i, item in enumerate(df["Subject"].unique()):
    colors[item] = lower + i

In [13]:
def color_map(item):
    return colors[item]

In [14]:
temp = df.loc[df["Experience"] != 'Land Revenue Mgmt & District Admn']
temp = temp.loc[temp["Experience"] != 'Personnel and General Administration']
temp = temp.loc[temp["Experience"] != 'Finance']

In [15]:
temp["text"] = temp["Count"].apply(lambda x: "size: "+str(x))

In [16]:
temp["colors"] = temp["Subject"].map(color_map)

In [17]:
temp = temp.loc[temp["Count"] > 0]

In [18]:
fig = go.Figure(data=[go.Scatter(
    y=temp["Experience"].to_list(),
    x=temp["Subject"].to_list(),
    text=temp["text"].to_list(),
    mode="markers",
    marker=dict(
        size=temp["Count"].apply(lambda x: x*100/max_value).to_list(),
        color = temp["colors"].to_list(),
    )
)])

fig.update_layout(
    autosize=False,
    height=1000,
    width=1400,
)

In [19]:
import plotly.express as px

In [24]:
admin_categories = [
    "Land Revenue Mgmt & District Admn",
    "Personnel and General Administration",
    "Finance"
]
def filter_by_value(df, col, value, include_admin=True, number_of_rows=5, percentage=False, include_other=False):

    df = df.loc[df[col].str.contains(value)]
    df.sort_values(["Count"], ascending=False, inplace=True)

    if include_admin and col == "Subject":
        for category in admin_categories:
            df = df.loc[df["Experience"] != category]

    temp = df[number_of_rows:]
    df = pd.DataFrame(df.head(number_of_rows))
    value_sum = temp["Count"].sum()
    
    if include_other:
        df = df.append({
            df.columns[0]: "Other",
            df.columns[1]: "Other",
            "Count": value_sum
        }, ignore_index=True)

    df.drop(columns=[col], inplace=True)
    df.set_index(df.columns[0], inplace=True) 

    if percentage:
        sum = df["Count"].sum()
        df = pd.DataFrame(df["Count"].apply(lambda x: (x / sum) * 100))

    try:
        return df.drop(columns=["text", "colors"])
    except:
        return df

In [30]:
filtered_df_subject = filter_by_value(df.copy(), 'Subject', "Engineering, Technology and Mathematics", include_admin=True, number_of_rows=15, percentage=False)

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
filtered_df_subject

,Count
Experience,
Urban Development,361
Industries,269
Agriculture & Cooperation,260
Human Resource Dev,253
Rural Dev,245
Energy,198
Home,194
Transport,188
Social Justice & Empowerment,170


In [40]:
fig = px.pie(
    filtered_df_subject, 
    names=filtered_df_subject.index, 
    values=filtered_df_subject["Count"], 
    title="Subjects",
    color_discrete_sequence=px.colors.qualitative.Bold
    )

In [44]:
fig = fig.update_traces(
    hoverinfo="label+percent",
    textinfo="value",
    insidetextfont=dict(
        color="white"
    ),
)

ValueError: Invalid property specified for object of type plotly.graph_objs.Pie: 'legend'

Did you mean "label0"?

    Valid properties:
        automargin
            Determines whether outside text labels can push the
            margins.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            customdata .
        direction
            Specifies the direction at which succeeding sectors
            follow one another.
        dlabel
            Sets the label step. See `label0` for more info.
        domain
            :class:`plotly.graph_objects.pie.Domain` instance or
            dict with compatible properties
        hole
            Sets the fraction of the radius to cut out of the pie